In [1]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

In [7]:
# %reload_ext autoreload
# %autoreload 2

# import numpy as np
# import pandas as pd
# from typing import Optional, List, Tuple, Callable
# from tqdm.notebook import tqdm
# from avatar.language import WranglingLanguage
# from avatar.analysis import *

In [2]:
titanic = pd.read_csv("../data/raw/demo/titanic.csv")
titanic.Survived = titanic.Survived.astype("category")
titanic

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [72]:
from avatar import bend_custom
from avatar.language import *
from avatar.selection import *
from avatar.analysis import *

In [91]:
# default language
language = WranglingLanguage()

# prune constants and identical columns
pruning = StackedFilter([MissingFilter(),
                         ConstantFilter(),
                         IdenticalFilter()])

# default preselection
preselection = StackedFilter([BijectiveFilter(),
                              UniqueFilter(),
                              CorrelationFilter()])

# feature selector
selection = SamplingSelector(
    iterations=200,
    evaluator=FeatureEvaluator(n_folds=4, max_depth=2, method=None)
)
# selection = CHCGASelector(
#     iterations=100,
#     evaluator=FeatureEvaluator(n_folds=5, max_depth=4, method=None)
# )

# dataset evaluator
evaluation = DatasetEvaluator(n_folds=5)

In [94]:
bend_custom(
    titanic,
    target="Survived",
    language=language,
    pruning=pruning,
    preselection=preselection,
    featureselection=selection,
    evaluation=evaluation
)

> Iteration 1
Starting with 12 features
Pruning; 11 left
Preselection; 9 left
Best features


,Sex,Pclass,Fare,Age,SibSp,Parch,Embarked,PassengerId,Survived
0,male,3,7.2500,22.0,1,0,S,1,0
1,female,1,71.2833,38.0,1,0,C,2,1
2,female,3,7.9250,26.0,0,0,S,3,1
3,female,1,53.1000,35.0,1,0,S,4,1
4,male,3,8.0500,35.0,0,0,S,5,0
...,...,...,...,...,...,...,...,...,...
886,male,2,13.0000,27.0,0,0,S,887,0
887,female,1,30.0000,19.0,0,0,S,888,1
888,female,3,23.4500,NaN,1,2,S,889,0
889,male,1,30.0000,26.0,0,0,C,890,1


Accuracy; 0.6095474232628209%
> Iteration 2
Starting with 118 features
Pruning; 70 left
Preselection; 33 left
Best features


,Sex,OneHot()(Pclass)_3,Pclass,Split( )(Name)_1,Fare,WordToNumber()(Ticket)_Ticket,ExtractNumber()(Ticket)_0,Age,Split( )(Name)_2,Parch,OneHot()(Parch)_0,SibSp,Survived
0,male,1,3,Mr.,7.2500,NaN,5.0,22.0,Owen,0,1,1,0
1,female,0,1,Mrs.,71.2833,NaN,17599.0,38.0,John,0,1,1,1
2,female,1,3,Miss.,7.9250,NaN,2.0,26.0,Laina,0,1,0,1
3,female,0,1,Mrs.,53.1000,113803.0,113803.0,35.0,Jacques,0,1,1,1
4,male,1,3,Mr.,8.0500,373450.0,373450.0,35.0,William,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,male,0,2,Rev.,13.0000,211536.0,211536.0,27.0,Juozas,0,1,0,0
887,female,0,1,Miss.,30.0000,112053.0,112053.0,19.0,Margaret,0,1,0,1
888,female,1,3,Miss.,23.4500,NaN,6607.0,NaN,Catherine,2,0,1,0
889,male,0,1,Mr.,30.0000,111369.0,111369.0,26.0,Karl,0,1,0,1


Accuracy; 0.6431611323834033%
> Iteration 3
Starting with 1426 features
Pruning; 381 left
Preselection; 75 left
Best features


,Sex,"OneHot()(ExtractWord([Miss, Mrs, Rev, Mr, Dr, Master])(Name)_0)_Mr","OneHot()(ExtractWord([Miss, Mrs, Rev, Mr, Dr, Master])(Name)_0)_Mrs",Pclass,OneHot()(Pclass)_3,Fare,"ExtractWord([Miss, Mrs, Rev, Mr, Dr, Master])(Split() )(Name)_0)_0","OneHot()(ExtractWord([Miss, Mrs, Rev, Mr, Dr, Master])(Name)_0)_Miss","ExtractWord([Miss, Mrs, Dr, Mr, Rev, Master])(Split(-)(Name)_0)_0","ExtractWord([Miss, Mrs, Rev, Mr, O, Dr, Master])(Split(')(Name)_0)_0",WordToNumber()(Ticket)_Ticket,ExtractNumber()(Split(/)(Ticket)_0)_0,Survived
0,male,1,0,3,1,7.2500,Mr,0,Mr,Mr,NaN,NaN,0
1,female,0,1,1,0,71.2833,Mrs,0,Mrs,Mrs,NaN,17599.0,1
2,female,0,0,3,1,7.9250,Miss,1,Miss,Miss,NaN,NaN,1
3,female,0,1,1,0,53.1000,Mrs,0,Mrs,Mrs,113803.0,113803.0,1
4,male,1,0,3,1,8.0500,Mr,0,Mr,Mr,373450.0,373450.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,male,0,0,2,0,13.0000,Rev,0,Rev,Rev,211536.0,211536.0,0
887,female,0,0,1,0,30.0000,Miss,1,Miss,Miss,112053.0,112053.0,1
888,female,0,0,3,1,23.4500,Miss,1,Miss,Miss,NaN,NaN,0
889,male,1,0,1,0,30.0000,Mr,0,Mr,Mr,111369.0,111369.0,1


Accuracy; 0.7687966857071119%


In [87]:
%debug

> /Users/gust/.local/share/virtualenvs/avatar-svm7aGJE/lib/python3.8/site-packages/sklearn/utils/validation.py(658)check_array()
    656         n_features = array.shape[1]
    657         if n_features < ensure_min_features:
--> 658             raise ValueError("Found array with %d feature(s) (shape=%s) while"
    659                              " a minimum of %d is required%s."
    660                              % (n_features, array.shape, ensure_min_features,

ipdb> up
> /Users/gust/.local/share/virtualenvs/avatar-svm7aGJE/lib/python3.8/site-packages/sklearn/utils/validation.py(72)inner_f()
     70                           FutureWarning)
     71         kwargs.update({k: arg for k, arg in zip(sig.parameters, args)})
---> 72         return f(**kwargs)
     73     return inner_f
     74 

ipdb> up
> /Users/gust/.local/share/virtualenvs/avatar-svm7aGJE/lib/python3.8/site-packages/sklearn/base.py(429)_validate_data()
    427                 # :(
    428                 check_X_param

ipdb> mask
array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
ipdb> code
array([[-1.,  1., -1., -1., -1., -1., -1., -1., -1., -1.]])
ipdb> quit
